In [1]:
import json
import pandas as pd

In [2]:
data = pd.read_csv('../../data/data.csv')

In [3]:
with open('../../data/id_mappings/floorId_to_mapId.json') as f:
    floorId_to_mapId = json.load(f)

In [4]:
map_id = floorId_to_mapId[str(1654)]

data = data[data['map_id'] == map_id]

macs = data['mac'].unique()
df = data[data['mac'] == macs[6]]

df = df.reset_index(drop=True)

print(len(df))
df.head()

139


,timestamp,mac,map_id,rssi,site_id,type,x,y
0,1.723558e+09,a9a7f864b741b8cbdbe9bc22cafcd3ef220bef6f56a6f9...,84e4e427-4160-45cf-85cf-940f6bada947,-58.10,48b01af6-138b-465d-8996-bace824f5726,wifi,13.725521,64.835556
1,1.723558e+09,a9a7f864b741b8cbdbe9bc22cafcd3ef220bef6f56a6f9...,84e4e427-4160-45cf-85cf-940f6bada947,-59.65,48b01af6-138b-465d-8996-bace824f5726,wifi,14.282210,65.997240
2,1.723558e+09,a9a7f864b741b8cbdbe9bc22cafcd3ef220bef6f56a6f9...,84e4e427-4160-45cf-85cf-940f6bada947,-61.10,48b01af6-138b-465d-8996-bace824f5726,wifi,14.880072,67.244840
3,1.723559e+09,a9a7f864b741b8cbdbe9bc22cafcd3ef220bef6f56a6f9...,84e4e427-4160-45cf-85cf-940f6bada947,-57.85,48b01af6-138b-465d-8996-bace824f5726,wifi,13.642944,64.663240
4,1.723556e+09,a9a7f864b741b8cbdbe9bc22cafcd3ef220bef6f56a6f9...,84e4e427-4160-45cf-85cf-940f6bada947,-64.10,48b01af6-138b-465d-8996-bace824f5726,wifi,16.402065,70.420900


In [10]:
import matplotlib.pyplot as plt

from IPython.display import HTML
import ipywidgets as widgets
from ipywidgets import interact

import sys
sys.path.append('../../src')

from device import Device

import numpy as np

In [11]:
with open('../../data/zValue_to_pValue.json') as f:
    zValue_to_pValue = json.load(f)
    zValue_to_pValue = {float(k): v for k, v in zValue_to_pValue.items()}

In [12]:
def update(index):
    curr_df = df.iloc[:index]
    device = Device(curr_df['mac'][0])


    for i in range(len(curr_df)):
        device.add_data(
            x = curr_df['x'][i],
            y = curr_df['y'][i],
            rssi = curr_df['rssi'][i],
            timestamp = curr_df['timestamp'][i],
            floor_id=curr_df['map_id'][i]
        )

    device.update_position(zValue_to_pValue=zValue_to_pValue)

    positions = np.array(device.positions)
    xs = positions[:, 0]
    ys = positions[:, 1]


    fig, ax = plt.subplots(figsize=(10, 10))

    ax.scatter(xs, ys, c='gray')
    ax.scatter(xs[-1], ys[-1], c='black')

    ax.scatter(device.x, device.y, c='red', marker='x')
    ax.add_patch(plt.Circle((device.x, device.y), device.error, color='red', alpha=0.3))


In [13]:
interact(update, index=widgets.IntSlider(min=0, max=len(df)-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='index', max=138), Output()), _dom_classes=('widget-inter…

<function __main__.update(index)>